In [44]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split,cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score, plot_confusion_matrix, classification_report,balanced_accuracy_score



In [28]:
# Algorithms
from sklearn.ensemble      import RandomForestClassifier
from sklearn.ensemble      import ExtraTreesClassifier
from sklearn.ensemble      import AdaBoostClassifier
from xgboost               import XGBClassifier


C:\Users\T430\anaconda3\envs\strive\lib\site-packages\xgboost\compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [12]:
df = pd.read_csv('dataset_5secondWindow.csv', index_col=0)
df

,id,time,activityrecognition#0,activityrecognition#1,android.sensor.accelerometer#mean,android.sensor.accelerometer#min,android.sensor.accelerometer#max,android.sensor.accelerometer#std,android.sensor.game_rotation_vector#mean,android.sensor.game_rotation_vector#min,...,sound#mean,sound#min,sound#max,sound#std,speed#mean,speed#min,speed#max,speed#std,target,user
0,16170,78.0,NaN,100.0,9.811476,9.758895,9.849411,0.014626,0.029340,0.029014,...,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000,Still,U12
1,15871,145.0,NaN,100.0,9.939207,7.707437,17.146631,1.775944,0.999925,0.999903,...,89.200210,89.065143,89.335277,0.191013,16.539349,16.539349,16.539349,0.628595,Car,U12
2,16811,150.0,NaN,100.0,9.827178,9.804817,9.849262,0.011199,0.665215,0.665213,...,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000,Still,U12
3,15831,105.0,NaN,77.0,9.673039,7.659674,12.304298,0.862553,0.996221,0.993781,...,87.470377,87.470377,87.470377,2.284186,17.739895,17.739895,17.739895,0.628595,Car,U12
4,876,77.0,NaN,100.0,9.993466,8.965621,10.891645,0.504117,0.563792,0.521799,...,89.770732,89.770732,89.770732,0.006389,9.000000,9.000000,9.000000,NaN,Car,U1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5888,14125,4.0,NaN,100.0,10.066610,4.564648,13.414389,3.373972,0.986319,0.974523,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Walking,U2
5889,19582,32.0,NaN,10.0,9.964271,8.768007,11.454213,0.665529,0.648602,0.648343,...,NaN,NaN,NaN,NaN,31.960054,31.960054,31.960054,NaN,Train,U3
5890,16178,86.0,NaN,100.0,9.812893,9.772881,9.862587,0.017633,0.024392,0.024081,...,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000,Still,U12
5891,10973,199.0,NaN,100.0,9.831144,9.807347,9.874450,0.013485,0.481212,0.481209,...,NaN,NaN,NaN,NaN,1.090415,1.090415,1.090415,NaN,Still,U10


## Separating the data base on users 

In [13]:
df_copy = df.copy()


In [14]:
X ,y= df_copy.drop(['target'],axis=1), df_copy['target']

In [18]:
y = y.array.reshape(-1, 1)

In [19]:
#Encoding the target variables 
from sklearn.preprocessing import OrdinalEncoder
oe = OrdinalEncoder()
oe.fit(y)

OrdinalEncoder()

In [22]:
Y = oe.transform(y)


In [ ]:
# Splitting the data
X_train,X_test,Y_train,Y_test = train_test_split(X,y,test_size=0.2,stratify=y, random_state=0)

In [ ]:
#scaling the data
data_prep = ColumnTransformer([('num_prepo', StandardScaler(), X.columns)],remainder='passthrough')

In [34]:
#Defining the models 
model_classifiers = {
  "Extra Trees": ExtraTreesClassifier(random_state=0),
  "Random Forest":RandomForestClassifier(random_state=0),
  "AdaBoost": AdaBoostClassifier(random_state=0),
  "XGBoost": XGBClassifier()}

model_classifiers_pipe = {name: make_pipeline(data_prep, model) for name, model in model_classifiers.items()}

## Benchmark accuracy

In [41]:
import time

In [ ]:
results = pd.DataFrame({'Model': [], 'Accuracy': [], 'Mean_cross_val':[], "Balanced Accuracy score":[],'Time': []})

for model_name, model in model_classifiers_pipe.items():
    start_time = time.time()
    
    # Training
    model.fit(X_train, Y_train)
    
    # Prediction
    pred = model.predict(X_test)

    total_time = time.time() - start_time # Time taken to fit and predict

    model_results = pd.DataFrame({"Model":    [model_name],
                              "Accuracy": [accuracy_score(Y_test, pred)*100],
                              "Mean_cross_val": cross_val_score(model, X_train, Y_train).mean(),
                              "Balanced Accuracy score": balanced_accuracy_score(Y_test, pred)*100,
                              "Time":     [total_time]})
    results = pd.concat([results, model_results])
    
    

results_ord = results.sort_values(by=['Accuracy'], ascending=False, ignore_index=True)
results_ord